In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import datetime
 
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('data/',one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
x = tf.placeholder("float",shape = [None,28,28,1])   #28*28*1 不是784个像素点
y_ = tf.placeholder("float",shape = [None,10])
'''
tf.reset_default_graph()函数:
用于清除默认图形堆栈并重置全局默认图形.默认图形是当前线程的一个属性。
该tf.reset_default_graph函数只适用于当前线程。
当一个tf.Session或者tf.InteractiveSession激活时调用这个函数会导致未定义的行为。
调用此函数后使用任何以前创建的tf.Operation或tf.Tensor对象将导致未定义的行为。
'''

C:\Users\77699\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


'\ntf.reset_default_graph()函数:\n用于清除默认图形堆栈并重置全局默认图形.默认图形是当前线程的一个属性。\n该tf.reset_default_graph函数只适用于当前线程。\n当一个tf.Session或者tf.InteractiveSession激活时调用这个函数会导致未定义的行为。\n调用此函数后使用任何以前创建的tf.Operation或tf.Tensor对象将导致未定义的行为。\n'

In [6]:
#定义卷积核  5*5的卷积核 1维的原始输入图像 输出32通道 32种不同的核进行卷积得到32个不同的特征
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32], stddev=0.1))
#偏置参数 用任意一个常数进行初始化
b_conv1 = tf.Variable(tf.constant(.1,shape = [32]))
#filter：卷积核 strides:步长 4个1 batchsize,h滑多少 ,w滑多少 ,通道数   SAME：可以自动补充
h_conv1 = tf.nn.conv2d(input = x,filter = W_conv1,strides = [1,1,1,1],padding = 'SAME') + b_conv1
#卷积一次 进行一次映射
h_conv1 = tf.nn.relu(h_conv1)  
#池化层 不会改变特征的个数
#没有参数 ksize:batchsize,2*2区域,通道数 strides:步长 4个1 batchsize,h滑多少 ,w滑多少 ,通道数
h_pool1 = tf.nn.max_pool(h_conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [7]:
'''
truncated_normal()  函数：
从截断的正态分布中取随机值。
生成的值服从具有指定平均值和标准偏差的正态分布，如果生成的值大于平均值两个标准差的值则丢弃重新选择。

tf.nn.conv2d(imput,filter,strides,padding,use_cudnn_on_gpu=None,name=None)函数：
实现卷积的函数，搭建卷积神经网络的核心方法
input:输入的图像
filter:相当于CNN中的卷积核
strides:卷积时在图像上的每一维的步长，[1,strides,strides,1]。
padding:设定卷积方式
use_cudnn_on_gpu:bool类型，是否使用cudnn加速，默认为true
返回值：feature map,shape为[batch,h,w,channels]

tf.nn.max_pool(value,ksize,strides,padding,name=None)函数：
CNN中最大池化操作
value:池化的输入，通常为feater map,
ksize:池化窗口大小，一般为[1,h,w,1]
strides:窗口在图像上的每一维的步长，[1,strides,strides,1]。
padding:设置池化方式
'''

'\ntruncated_normal()\xa0 函数：\n从截断的正态分布中取随机值。\n生成的值服从具有指定平均值和标准偏差的正态分布，如果生成的值大于平均值两个标准差的值则丢弃重新选择。\n\ntf.nn.conv2d(imput,filter,strides,padding,use_cudnn_on_gpu=None,name=None)函数：\n实现卷积的函数，搭建卷积神经网络的核心方法\ninput:输入的图像\nfilter:相当于CNN中的卷积核\nstrides:卷积时在图像上的每一维的步长，[1,strides,strides,1]。\npadding:设定卷积方式\nuse_cudnn_on_gpu:bool类型，是否使用cudnn加速，默认为true\n返回值：feature map,shape为[batch,h,w,channels]\n\ntf.nn.max_pool(value,ksize,strides,padding,name=None)函数：\nCNN中最大池化操作\nvalue:池化的输入，通常为feater map,\nksize:池化窗口大小，一般为[1,h,w,1]\nstrides:窗口在图像上的每一维的步长，[1,strides,strides,1]。\npadding:设置池化方式\n'

In [8]:
# 第二个卷积层 也可以写成函数
def conv2d(x,W):
	return tf.nn.conv2d(input = x, filter = W, strides = [1,1,1,1],padding = 'SAME')
def max_pool_2x2(x):	
	return tf.nn.max_pool(x,ksize=[1,2,2,1],strides = [1,2,2,1],padding = 'SAME')
#第二个卷积核
W_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(.1,shape = [64]))
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
#第一个全连接层
W_fcl = tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.1))	#7*7*64转换为1024个特征
b_fcl = tf.Variable(tf.constant(.1,shape = [1024]))
#reshape将7*7*64拉伸成一长条,即h_pool2   -1自动取了一个合适的值，1
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fcl = tf.nn.relu(tf.matmul(h_pool2_flat,W_fcl) + b_fcl)

In [11]:
'''
tf.reshape(tensor,shape,name=None)函数：
将tensor变化为shape的形式，shape为一个列表形式。
防止过拟合
'''
#droput layer:杀死一部分神经元 防止过拟合
#指定保留率
keep_prob = tf.placeholder("float")
h_fcl_drop = tf.nn.dropout(h_fcl,keep_prob)

In [12]:
'''
tf.nn.dropout(x,keep_prob)函数
防止过拟合，会随机扔掉一部分神经元。
x:输入
keep_prob:设置神经元被选中的概率，在初始化时keep_prob是一个占位符。
'''
#第二个全连接层
W_fc2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))	#
b_fc2 = tf.Variable(tf.constant(.1,shape = [10]))

In [13]:
#最后一层
y = tf.matmul(h_fcl_drop,W_fc2) + b_fc2

In [14]:
#求损失值，指定优化器

#用损失函数求交叉熵的损失
crossEntropyLoss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_,logits = y))
#指定优化器 AdamOptimizer可以自适应的调整学习率，比梯度下降好
trainStep = tf.train.AdamOptimizer().minimize(crossEntropyLoss)
#计算正确率
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
#开始训练
sess.run(tf.global_variables_initializer())
batchsize = 50
for i in range(5000):
    batch = mnist.train.next_batch(batchsize)
    trainingInputs = batch[0].reshape([batchsize,28,28,1])		#输入大小必须为28*28*1
    trainingLabels = batch[1]
    if i%100 == 0:
        trainAccuracy  = accuracy.eval(session = sess,feed_dict = {x:trainingInputs,y_:trainingLabels,keep_prob:0.5})
        print("step %d, training accuracu %g"%(i,trainAccuracy))
    trainStep.run(session = sess,feed_dict={x:trainingInputs,y_:trainingLabels,keep_prob:0.5})

step 0, training accuracu 0.14
step 100, training accuracu 0.92
step 200, training accuracu 0.96
step 300, training accuracu 0.96
step 400, training accuracu 0.98
step 500, training accuracu 0.94
step 600, training accuracu 0.96
step 700, training accuracu 0.98
step 800, training accuracu 0.96
step 900, training accuracu 0.98
step 1000, training accuracu 0.98
step 1100, training accuracu 1
step 1200, training accuracu 0.98
step 1300, training accuracu 0.96
step 1400, training accuracu 0.98
step 1500, training accuracu 1
step 1600, training accuracu 1
step 1700, training accuracu 0.98
step 1800, training accuracu 0.9
step 1900, training accuracu 1
step 2000, training accuracu 0.98
step 2100, training accuracu 1
step 2200, training accuracu 1
step 2300, training accuracu 0.98
step 2400, training accuracu 0.98
step 2500, training accuracu 1
step 2600, training accuracu 0.98
step 2700, training accuracu 1
step 2800, training accuracu 1
step 2900, training accuracu 1
step 3000, training acc